# Handle Java Compilation Errors

Explore how we would want to accept compilation errors, and how we would want the agent to get the updates that need to be made

## Setting up the environment

In [2]:
# Install local kai package in the current Jupyter kernel
import sys

!{sys.executable} -m pip install -e ../../

Obtaining file:///Users/shurley/repos/kai/kai
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/fabianvf/vcrpy.git (to revision httpx-async-threadpool) to /private/var/folders/vt/5bfp7vyd1h79_7k5ygr0fttr0000gn/T/pip-install-iekowe1p/vcrpy_6be2b4ed9a1b40aaa56d5df6340275f9
  Running command git clone --filter=blob:none --quiet https://github.com/fabianvf/vcrpy.git /private/var/folders/vt/5bfp7vyd1h79_7k5ygr0fttr0000gn/T/pip-install-iekowe1p/vcrpy_6be2b4ed9a1b40aaa56d5df6340275f9
  Running command git checkout -b httpx-async-threadpool --track origin/httpx-async-threadpool
  Switched to a new branch 'httpx-async-threadpool'
  branch 'httpx-async-threadpool' set up to track 'origin/httpx-async-threadpool'.
  Resolved https://github.com/fabianvf/vcrpy.git to commit db764f025eaad9459e1553da36fd71400a0c9849
 

In [3]:
import pprint

pp = pprint.PrettyPrinter(indent=2)

## Create an Base Agent type and an specific Compilation Agent Type

In [14]:
from typing import Any, Dict, List, Optional, Protocol, runtime_checkable
from dataclasses import dataclass
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import HumanMessage, SystemMessage, BaseMessage
from jinja2 import Template

@dataclass
class TaskRequest:
    """(Experimental Task Request)

    A simple TaskRequest to just get the TaskAgent Protocal figured out.
    """

    type_of_chat: str = None
    compile_errors: List[str] = None
    file_path: str = None

@runtime_checkable
class TaskAgent(Protocol):
    """A protocol for Task Agents to be used with the tasking system in Kai

    An agent is responsible for doing a concreate piece of fixing of a java client
    by using one or more LLM prompts and chats.
    """

    def accepts(self,task_request: TaskRequest) -> bool:
        """Does this agent Accept the task

        Args:
            task_request (TaskRequest): the type of task and task information.
        """ 

    async def run(self, task_request: TaskRequest) -> dict[str, any]:
        """Run the agent for the given TaskRequest
        """

class CompilationErrorAgent(TaskAgent):
    """An agent responsible for dealing with compilation errors.
    """

    system_message=SystemMessage(content="""
    I will give you compiler errors and the offending line of code, and you will need to use the file to determine how to fix them. You should only use compiler errors to determine what to fix.

    Make sure that the references to any changed types are kept.

    You must reason through the required changes and rewrite the Java file to make it compile. 

    You will then provide an step-by-step explaination of the changes required tso that someone could recreate it in a similar situation. 
    """)

    chat_message_template=Template("""
    [INST]
    ## Compile Errors
    {{compile_errors}}

    ## Input File
    {{src_file_contents}}


    # Ouput Instructions 
    Structure your output in Markdown format such as:

    ## Updated Java File
    Rewrite the java file here

    ## Reasoning 
    Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why. 

    ## Addition Infomation (optional)
    If you have additional details or steps that need to be perfomed, put it here. Say I have completed the changes when you are done explaining the reasoning[/INST]
    """
    )

    def __init__(
            self,
            name: str,
            llm: BaseChatModel,
    ):
        """docs"""
        self.name = name
        self.__llm = llm
    
    def accepts(self, task_request: TaskRequest) -> bool:
        if task_request.type_of_chat == "compilation":
            return True
        return False
    
    async def run(self, task_requst: TaskRequest) -> BaseMessage:
        with open(task_requst.file_path) as f: src_file_contents= f.read()

        compile_errors = ""
        for c in task_requst.compile_errors:
            compile_errors = "\n".join([compile_errors, c])

        content = self.chat_message_template.render(
            src_file_contents=src_file_contents,
            compile_errors=compile_errors)

        aimessage = await self.__llm.ainvoke([self.system_message, HumanMessage(content=content)])
        return aimessage



## Run Prompt via a task.

Create the Agent and then create the task_request to get the result of the agent.

### Testing for the type of a field has changed for a given file

This will test that field change, gets the correct output (updating the getter and setter to the new Methods)

In [15]:
from kai.service.llm_interfacing.model_provider import ModelProvider
from kai.models.kai_config import KaiConfig
import os

config = KaiConfig.model_validate_filepath("../../kai/config.toml")
modelProvider = ModelProvider(config.models)
agent = CompilationErrorAgent("java_compilation_error", modelProvider.llm)

task_request = TaskRequest(type_of_chat="CompilationError", 
                           compile_errors=["Line of code: return itemId;\nError: incompatible types: java.util.UUID cannot be converted to java.lang.String",
                                           "Line of code: this.itemId = itemId;\nError: incompatible types: java.lang.String cannot be converted to java.util.UUID"],
                            file_path="./testing_field_type_change_errors/InventoryEntity.java")

o = await agent.run(task_requst=task_request)

if "public UUID getItemId()" in o.content:
    print("Succesfull")


## Updated Java File
```java
package com.redhat.coolstore.model;

import java.io.Serializable;
import java.util.UUID;

import javax.persistence.Column;
import javax.persistence.Entity;
import javax.persistence.Id;
import javax.persistence.Table;
import javax.persistence.UniqueConstraint;
import javax.xml.bind.annotation.XmlRootElement;

@Entity
@XmlRootElement
@Table(name = "INVENTORY", uniqueConstraints = @UniqueConstraint(columnNames = "itemId"))
public class InventoryEntity implements Serializable {

    private static final long serialVersionUID = 7526472295622776147L; 

    @Id
    private UUID itemId;


    @Column
    private String location;


    @Column
    private int quantity;


    @Column
    private String link;

    public InventoryEntity() {

    }

    public UUID getItemId() {
        return itemId;
    }

    public void setItemId(UUID itemId) {
        this.itemId = itemId;
    }

    public String getLocation() {
        return location;
    }

    public void set